In [2]:
import pandas as pd

In [19]:
df_sales_detail = pd.read_csv('../data/2022-04-01T12_df_sales_detail.csv')
df_sales_detail.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [22]:
df_sales = pd.read_csv('../data/2022-04-01T12_df_sales.csv')
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [23]:
df_web_logs = pd.read_csv('../data/2022-04-01T12_df_web_logs.csv')
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


## Топ-3 товаров по суммарной выручке

Определить топ-3 товара по суммарной выручке.

In [20]:
def get_top_item_revenue(data: pd.DataFrame, item_col: str, price_col: str, top_k: int) -> pd.DataFrame:
    data_revenue = data.groupby(item_col).agg(item_revenue=(price_col, 'sum')). \
            sort_values(by='item_revenue', ascending=False).reset_index()
    return data_revenue.iloc[:top_k]

In [21]:
get_top_item_revenue(df_sales_detail, item_col='good', price_col='price', top_k=3)

,good,item_revenue
0,chefs pizza,24558240
1,double pepperoni pizza,22558380
2,chicken bbq pizza,17622540


## Время от захода на сайт до покупки

Оценка среднего срока, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

In [25]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [24]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [69]:
def get_time_for_buy(data_sales: pd.DataFrame, data_web_logs: pd.DataFrame) -> int:
    user_sales = data_sales[['user_id', 'date']].rename(columns={'date': 'sale_date'})
    user_logs = data_web_logs[['user_id', 'date']].rename(columns={'date': 'event_date'})
    user_sales['sale_date'] = pd.to_datetime(user_sales['sale_date'])
    user_logs['event_date'] = pd.to_datetime(user_logs['event_date'])
    first_user_logs = user_logs.groupby('user_id')['event_date'].min().reset_index()

    user_sales_logs = user_sales.merge(right=first_user_logs, how='inner', on='user_id')
    user_sales_logs['diff_time'] = user_sales_logs['sale_date'] - user_sales_logs['event_date']
    user_sales_logs['diff_time_minute'] = user_sales_logs['diff_time'].dt.total_seconds() / 60
    user_buy_time = user_sales_logs[user_sales_logs['diff_time_minute'] <= 120]
    return round(user_buy_time['diff_time_minute'].mean())

In [70]:
get_time_for_buy(df_sales, df_web_logs)

17

## Удержание клиентов

Как много пользователей возвращается к нам из месяца в месяц. Какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

In [110]:
def retantion_next_month(data: pd.DataFrame, first_month: tuple[int, int], next_month: tuple[int, int]) -> float:
    users_data = data[['user_id', 'date']].copy()
    users_data['date'] = pd.to_datetime(users_data['date'])

    y1, m1 = first_month
    y2, m2 = next_month

    users_first_month = users_data[(users_data['date'].dt.year == y1) & (users_data['date'].dt.month == m1)]
    users_first_month = users_first_month[['user_id']].drop_duplicates()
    users_next_month = users_data[(users_data['date'].dt.year == y2) & (users_data['date'].dt.month == m2)]
    users_next_month = users_next_month[['user_id']].drop_duplicates()

    retantion = pd.merge(left=users_first_month, right=users_next_month, how='inner', on='user_id')
    return round(retantion['user_id'].count() / users_first_month['user_id'].count(), 2)

In [111]:
retantion_next_month(df_sales, first_month=(2022, 2), next_month=(2022, 3)) # 69662 

0.66